## AIHub Json Preprocessing

### Development Environment

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
%pip install pandas

In [ ]:
%pip install ray

In [1]:
import re
import os
import kss
import ray
import json
import time
from time import sleep
from tqdm import tqdm
from mecab import MeCab
import pandas as pd
from glob import glob
from itertools import chain

In [19]:
pwd

'd:\\AIHUB'

### AIHUB 기계독해

[Source](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=89)

#### Convert JSON File to TXT File

In [ ]:
from data_preprocessing import make_json_txt_file_path_list
from data_preprocessing import divide_source_file_list
from extract_source_text import make_sources

In [10]:
json_path_list = ['AIHUB_기계독해'+ '/**/*.json']
txt_path_list =  ["exploration/machine_reading_pro/AIHUB_machine_reading_"]

In [11]:
json_file_list, txt_file_path_list = \
    make_json_txt_file_path_list(json_path_list, txt_path_list)

The number of file: 3


In [12]:
source_file_index_df = pd.DataFrame(json_file_list, columns=['source_file_name'])
source_file_index_df.to_excel("source_file_index/machine_reading_source_file_index.xlsx", index=False)

In [14]:
def count_number_of_txt_file_with_batch_list(source_file_list, batch_size):

    source_file_by_batch_df = pd.DataFrame({'File':[0], 'Length of Source List':[0],
                                        'The Number of TXT File':[0], 
                                        'Description':[0]})
                                            
    the_number_of_total_txt_file = 0
    the_number_of_txt_file_list = []
    
    for i in range(len(source_file_list)):    
        
        source_file = source_file_list[i]   

        with open(source_file, 'r', encoding='utf-8') as one_json_file:
            one_json_sample = json.load(one_json_file)

        source_list = make_sources(one_json_sample)
        
        the_number_of_txt_file = ((len(source_list) // batch_size) + 1)

        if len(source_list) >= batch_size:
            source_file_by_batch_df.loc[i] = [source_file,
                                              len(source_list), the_number_of_txt_file, ""]
            the_number_of_txt_file_list.append(the_number_of_txt_file)
            the_number_of_total_txt_file  += the_number_of_txt_file

        elif len(source_list) < batch_size:
            source_file_by_batch_df.loc[i] = [source_file,
                                              len(source_list), the_number_of_txt_file,
                                              "not subject of batch. small source list."]
            the_number_of_txt_file_list.append(1)
            the_number_of_total_txt_file  += 1

    print("Batch Size:", batch_size)
    print("The number of txt file:", the_number_of_total_txt_file)

    if 'rain' in source_file:
        source_file_by_batch_df.to_excel("source_file_by_batch/machine_reading_train.xlsx", index=False)
    elif 'alid' in source_file:
        source_file_by_batch_df.to_excel("source_file_by_batch/machine_reading_valid.xlsx", index=False)
    else:
         source_file_by_batch_df.to_excel("source_file_by_batch/machine_reading.xlsx", index=False)

    return the_number_of_total_txt_file, the_number_of_txt_file_list

In [15]:
def write_jsontext_to_txt_file_with_batch_list(source_file_list,
                                    text_file_path_list,
                                    batch_size, the_number_of_txt_file_list):
  
  
  progress_length = sum(the_number_of_txt_file_list)
  print("[Size]")
  print("The number of preprocessing corpus: " + str(progress_length))
  print("\n[Order]")
  pbar = tqdm(range(progress_length))
  num = 0

  for i in range(len(source_file_list)):

    source_file = source_file_list[i]
    
    with open(source_file, 'r', encoding='utf-8') as one_json_file:
      one_json_sample = json.load(one_json_file)

    source_list = make_sources(one_json_sample)
    
    n = batch_size
    source_batch_list = list(divide_source_file_list(source_list, n))
      
    for source_list in source_batch_list:
        with open(os.path.join('AIHUB_corpus/' + text_file_path_list[i][:-4] + "_" + str(num) + ".txt"), "a", encoding='utf-8') as fp:        
          fp.write("\n".join(source_list)) 
        num += 1  
        pbar.n += 1
        pbar.refresh()
        time.sleep(0.01)
  pbar.close()  

In [16]:
batch_size = 1000
the_number_of_txt_file, the_number_of_txt_file_list = count_number_of_txt_file_with_batch_list(json_file_list, batch_size)

Batch Size: 1000
The number of txt file: 104


In [18]:
source_file_by_batch_df = pd.read_excel('source_file_by_batch/machine_reading.xlsx', engine='openpyxl')  
source_file_by_batch_df

,Unnamed: 0,File,Length of Source List,The Number of txt File,Description
0,0,AIHUB_기계독해\ko_nia_normal_squad_all.json ~ AI...,47314,48,NaN
1,1,AIHUB_기계독해\ko_nia_clue0529_squad_all.json ~ ...,34500,35,NaN
2,2,AIHUB_기계독해\ko_nia_noanswer_squad_all.json ~ ...,20030,21,NaN


In [19]:
batch_size = 1000
write_jsontext_to_txt_file_with_batch_list(json_file_list, txt_file_path_list,
                batch_size, the_number_of_txt_file_list)

[Size]
The number of preprocessing corpus: 104

[Order]


100%|██████████| 104/104 [00:20<00:00,  5.14it/s]


#### Preprocess TXT File

In [ ]:
from sentence_segmentation import preprocessing_text
from data_preprocessing import make_pro_post_txt_file_path_list
from data_preprocessing import merge_and_deduplicate_corpus_txt
from reading_data import reading_txt

In [18]:
pro_corpus_path = "AIHUB_corpus/exploration/machine_reading_pro/AIHUB_machine_reading_" + "*.txt"
pro_total_corpus_path_list, post_total_corpus_path_list = make_pro_post_txt_file_path_list(pro_corpus_path)

In [19]:
len(pro_total_corpus_path_list)

104

In [20]:
pro_coprus_file = pro_total_corpus_path_list[0]
line_length = 1
data_type = "source"

reading_txt(pro_coprus_file, line_length, data_type)

한국청소년단체협의회와 여성가족부는 22일부터 28일까지 서울과 충북 괴산에서 '국제청소년포럼'을 연다고 21일 밝혔다. 한국 미국 캐나다 호주 등 전 세계 32개국 75여명의 대학생, 청소년들이 모여 전 세계적 현안문제에 대한 대안과 해결책을 모색하는 자리다. 이번 포럼의 주제는 '청소년과 뉴미디어'다. 스마트폰 SNS 태블릿PC 등 새로운 커뮤니케이션 매체인 '뉴미디어'에 대한 성찰과 문제점에 대해 토론한다. 기조강연을 시작으로 국가별 주제관련 사례발표, 그룹 토론 및 전체총회, '청소년선언문' 작성 및 채택 등 다양한 프로그램을 운영한다. 개회식은 22일 서울 방화동에 있는 국제청소년센터 국제회의장에서 한다. 전 세계 32개국 대학생ㆍ청소년 참가자와 전국의 청소년기관단체장과 청소년지도자 여성가족부 주한외교사절 등 100여명이 참석할 예정이다. 23일에는 유엔미래포럼 박영숙 대표가 '뉴미디어의 균형 있는 발전을 위한 청소년의 역할'에 대해 기조강연을 한다. 뉴미디어의 올바른 활용방안과 청소년문화의 형성에 대해 설명할 계획이다. 27일 폐회식에서는 '청소년선언문'을 채택한다. 선언문에는 전 세계적으로 뉴미디어의 바람직한 발전을 촉구하며 각국 청년들이 함께 실천할 수 있는 내용 등이 담길 예정이다. 한국청소년단체협의회는 포럼이 끝난 뒤 UN 등 국제기구와 참가자 각국 정부 등 국제사회에 선언문을 전달할 예정이다.



In [21]:
pro_coprus_file = pro_total_corpus_path_list[0]
line_length = 1
data_type = "preprocessing"

reading_txt(pro_coprus_file, line_length, data_type)

한국청소년단체협의회와 여성가족부는 22일부터 28일까지 서울과 충북 괴산에서  국제청소년포럼 을 연다고 21일 밝혔다.

한국 미국 캐나다 호주 등 전 세계 32개국 75여명의 대학생, 청소년들이 모여 전 세계적 현안문제에 대한 대안과 해결책을 모색하는 자리다.

스마트폰 SNS 태블릿PC 등 새로운 커뮤니케이션 매체인  뉴미디어 에 대한 성찰과 문제점에 대해 토론한다.

기조강연을 시작으로 국가별 주제관련 사례발표, 그룹 토론 및 전체총회,  청소년선언문  작성 및 채택 등 다양한 프로그램을 운영한다.

개회식은 22일 서울 방화동에 있는 국제청소년센터 국제회의장에서 한다.

전 세계 32개국 대학생 청소년 참가자와 전국의 청소년기관단체장과 청소년지도자 여성가족부 주한외교사절 등 100여명이 참석할 예정이다.

23일에는 유엔미래포럼 박영숙 대표가  뉴미디어의 균형 있는 발전을 위한 청소년의 역할 에 대해 기조강연을 한다.

뉴미디어의 올바른 활용방안과 청소년문화의 형성에 대해 설명할 계획이다.

선언문에는 전 세계적으로 뉴미디어의 바람직한 발전을 촉구하며 각국 청년들이 함께 실천할 수 있는 내용 등이 담길 예정이다.

한국청소년단체협의회는 포럼이 끝난 뒤 UN 등 국제기구와 참가자 각국 정부 등 국제사회에 선언문을 전달할 예정이다.



In [22]:
ray.init(num_cpus = 4)

@ray.remote
def ray_preprocessing_text(source, corpus_path):

    preprocessing_sentence_list = preprocessing_text(source, corpus_path)

    return preprocessing_sentence_list

2023-07-11 10:42:24,945	INFO worker.py:1625 -- Started a local Ray instance.


In [23]:
def preprocessing_corpus_txt(pro_total_corpus_path_list, post_total_corpus_path_list):
    
    progress_length = len(pro_total_corpus_path_list)
    print("[Size]")
    print("The number of preprocessing corpus: " + str(progress_length))
    print("\n[Order]")
    pbar = tqdm(range(progress_length))
    process_num = 10    

    for pro, post in zip(pro_total_corpus_path_list, post_total_corpus_path_list):

        sentence_list = []

        with open(pro, 'r', encoding='utf-8') as f:
            lines = f.read().splitlines() 
            nested_lines_num = len(lines) // process_num
            for i in range(nested_lines_num - 1):
                start_line = process_num * i
                end_line = process_num * (i+1)
                futures = [ray_preprocessing_text.remote(lines[start_line:end_line][j], pro) for j in range(process_num)]
                results = ray.get(futures)

                if i == nested_lines_num - 2:
                    futures = [ray_preprocessing_text.remote(lines[end_line:][j], pro) for j in range(len(lines) - end_line)]
                    results = ray.get(futures)

                sentences = list(chain.from_iterable(results))
                sentence_list.append(sentences)

        sentence_list = list(chain.from_iterable(sentence_list))

        with open(post, 'a', encoding='utf-8') as fp:
            fp.write("\n".join(sentence_list))

        pbar.n += 1
        pbar.refresh()
        time.sleep(0.01)

    pbar.close() 

In [24]:
preprocessing_corpus_txt(pro_total_corpus_path_list, post_total_corpus_path_list)

[Size]
The number of preprocessing corpus: 104

[Order]


 96%|█████████▌| 100/104 [1:16:56<03:04, 46.16s/it](formal_preprocessing_text pid=19244) [Korean Sentence Splitter]: Too long text! turn off quotes calibration!
(formal_preprocessing_text pid=18688) [Korean Sentence Splitter]: Too long text! turn off quotes calibration!
100%|██████████| 104/104 [1:18:44<00:00, 45.43s/it]


In [223]:
ray.shutdown()

In [ ]:
preprocessing_corpus_path = "AIHUB_corpus/exploration/machine_reading_post/AIHUB_machine_reading_" +"*.txt"
merge_corpus_path = 'AIHUB_corpus/duplicate/AIHUB_machine_reading.txt'
deduplicate_corpus_path = 'AIHUB_corpus/AIHUB_machine_reading.txt'

In [ ]:
merge_and_deduplicate_corpus_txt(preprocessing_corpus_path, merge_corpus_path, 
                                  deduplicate_corpus_path)